In [0]:
dbutils.fs.head("/FileStore/tables/clinicaltrial_2021.csv/clinicaltrial_2021.csv")

[Truncated to first 65536 bytes]
Out[1]: "Id|Sponsor|Status|Start|Completion|Type|Submission|Conditions|Interventions\r\nNCT02758028|The University of Hong Kong|Recruiting|Aug 2005|Nov 2021|Interventional|Apr 2016||\r\nNCT02751957|Duke University|Completed|Jul 2016|Jul 2020|Interventional|Apr 2016|Autistic Disorder,Autism Spectrum Disorder|\r\nNCT02758483|Universidade Federal do Rio de Janeiro|Completed|Mar 2017|Jan 2018|Interventional|Apr 2016|Diabetes Mellitus|\r\nNCT02759848|Istanbul Medeniyet University|Completed|Jan 2012|Dec 2014|Observational|May 2016|Tuberculosis,Lung Diseases,Pulmonary Disease|\r\nNCT02758860|University of Roma La Sapienza|Active, not recruiting|Jun 2016|Sep 2020|Observational [Patient Registry]|Apr 2016|Diverticular Diseases,Diverticulum,Diverticulosis|\r\nNCT02757209|Consorzio Futuro in Ricerca|Completed|Apr 2016|Jan 2018|Interventional|Apr 2016|Asthma|Fluticasone,Xhance,Budesonide,Formoterol Fumarate,Salmeterol Xinafoate\r\nNCT02752438|Ankara University|Unkn

In [0]:


clinicaltrial_2021 = spark.read.csv("/FileStore/tables/clinicaltrial_2021.csv/clinicaltrial_2021.csv", header=True, inferSchema=True, sep = '|')


In [0]:
clinicaltrial_2021.show(5)

+-----------+--------------------+--------------------+--------+----------+--------------------+----------+--------------------+-------------+
|         Id|             Sponsor|              Status|   Start|Completion|                Type|Submission|          Conditions|Interventions|
+-----------+--------------------+--------------------+--------+----------+--------------------+----------+--------------------+-------------+
|NCT02758028|The University of...|          Recruiting|Aug 2005|  Nov 2021|      Interventional|  Apr 2016|                null|         null|
|NCT02751957|     Duke University|           Completed|Jul 2016|  Jul 2020|      Interventional|  Apr 2016|Autistic Disorder...|         null|
|NCT02758483|Universidade Fede...|           Completed|Mar 2017|  Jan 2018|      Interventional|  Apr 2016|   Diabetes Mellitus|         null|
|NCT02759848|Istanbul Medeniye...|           Completed|Jan 2012|  Dec 2014|       Observational|  May 2016|Tuberculosis,Lung...|         null|

In [0]:
clinicaltrial_2021.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Sponsor: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Start: string (nullable = true)
 |-- Completion: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Submission: string (nullable = true)
 |-- Conditions: string (nullable = true)
 |-- Interventions: string (nullable = true)



In [0]:
#ques 1:The number of studies in the dataset
clinicaltrial_2021.distinct().count()

Out[5]: 387261

In [0]:
#ques 2: Types of studies and their frequencies
from pyspark.sql.functions import col
type_counts = clinicaltrial_2021.groupby('Type').count()
type_counts = type_counts.select(col('Type'), col('count').alias('2021'))
type_counts = type_counts.orderBy('2021', ascending=False)

type_counts.show()


+--------------------+------+
|                Type|  2021|
+--------------------+------+
|      Interventional|301472|
|       Observational| 77540|
|Observational [Pa...|  8180|
|     Expanded Access|    69|
+--------------------+------+



In [0]:
#ques 3: Top 5 condition with their frequency
from pyspark.sql.functions import col, trim, split, explode
conditions_df = clinicaltrial_2021.select(explode(split(col("Conditions"), ",")).alias("Condition")).select(trim(col("Condition")).alias("Condition"))
condition_count = conditions_df.groupBy(col("Condition")).count()
top_conditions = condition_count.orderBy(col("count").desc()).limit(5)
top_conditions.show() 

+-----------------+-----+
|        Condition|count|
+-----------------+-----+
|        Carcinoma|13389|
|Diabetes Mellitus|11080|
|        Neoplasms| 9371|
| Breast Neoplasms| 8640|
|         Syndrome| 8032|
+-----------------+-----+



In [0]:
#ques 4 prepration
#importing pharma data

pharma = spark.read.csv("/FileStore/tables/pharma.csv/pharma.csv", header=True, inferSchema=True, sep = ',')

In [0]:
pharma.show(5)

+--------------------+-------------------+--------------+------------------------+---------------------------------------------------------+------------+------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------+--------------------+------------------+---------------------+-----+-------+-----------------------------+------------------+--------------+--------+-------+----+----------+--------------------+--------------------+------------------+-------------------+---------------------------+------------------------+---------------------------+--------------------+--------------------+
|             Company|     Parent_Company|Penalty_Amount|Subtraction_From_Penalty|Penalty_Amount_Adjusted_For_Eliminating_Multiple_Counting|Penalty_Year|Penalty_Date|       Offense_Group|     Primary_Offense|   Secondary_Offense|         Description|Level_of_Government|  Action_Type|              Agency|    Civil/Criminal|Prosecution_Ag

In [0]:
pharma.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Parent_Company: string (nullable = true)
 |-- Penalty_Amount: string (nullable = true)
 |-- Subtraction_From_Penalty: string (nullable = true)
 |-- Penalty_Amount_Adjusted_For_Eliminating_Multiple_Counting: string (nullable = true)
 |-- Penalty_Year: integer (nullable = true)
 |-- Penalty_Date: integer (nullable = true)
 |-- Offense_Group: string (nullable = true)
 |-- Primary_Offense: string (nullable = true)
 |-- Secondary_Offense: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Level_of_Government: string (nullable = true)
 |-- Action_Type: string (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Civil/Criminal: string (nullable = true)
 |-- Prosecution_Agreement: string (nullable = true)
 |-- Court: string (nullable = true)
 |-- Case_ID: string (nullable = true)
 |-- Private_Litigation_Case_Title: string (nullable = true)
 |-- Lawsuit_Resolution: string (nullable = true)
 |-- Facility_State: string

In [0]:
#ques 4:10 most common sponsors that are not pharmaceutical companies, along with the number of clinical trials they have sponsored
from pyspark.sql.functions import col, collect_set
parent_company_set = set(pharma.select("Parent_Company").distinct().toPandas()["Parent_Company"].tolist())
clinicaltrial_2021 = clinicaltrial_2021.filter(~col("Sponsor").isin(parent_company_set))
sponsor_count = clinicaltrial_2021.groupBy("Sponsor").count()
top_sponsors = sponsor_count.orderBy(col("count").desc()).limit(10)
top_sponsors.show()

+--------------------+-----+
|             Sponsor|count|
+--------------------+-----+
|National Cancer I...| 3218|
|M.D. Anderson Can...| 2414|
|Assistance Publiq...| 2369|
|         Mayo Clinic| 2300|
|Merck Sharp & Doh...| 2243|
|   Assiut University| 2154|
|Novartis Pharmace...| 2088|
|Massachusetts Gen...| 1971|
|    Cairo University| 1928|
|   Hoffmann-La Roche| 1828|
+--------------------+-----+



In [0]:
#ques 5: Plot number of completed studies each month in a given year
clinicaltrial_2021.select("Status", "Completion").show()

+--------------------+----------+
|              Status|Completion|
+--------------------+----------+
|          Recruiting|  Nov 2021|
|           Completed|  Jul 2020|
|           Completed|  Jan 2018|
|           Completed|  Dec 2014|
|Active, not recru...|  Sep 2020|
|           Completed|  Jan 2018|
|      Unknown status|  Jul 2017|
|      Unknown status|  Nov 2019|
|           Completed|  Jul 2017|
|           Completed|  Jan 2021|
|           Completed|  Oct 2016|
|           Completed|  Dec 2015|
|           Completed|  Mar 2016|
|           Completed|  May 2019|
|Active, not recru...|  Dec 2021|
|      Unknown status|  Jun 2017|
|          Terminated|  Jul 2016|
|           Completed|  Oct 2019|
|      Unknown status|  Dec 2018|
|          Recruiting|  Oct 2022|
+--------------------+----------+
only showing top 20 rows



In [0]:
#ques 5 final output
from pyspark.sql.functions import *
clinicaltrial_2021 = spark.table("clinicaltrial_2021")
result_df = (clinicaltrial_2021
             .filter((col("Status") == "Completed") & (col("Completion").like("%2021%")))
             .select(substring(col("Completion"), 1, 3).alias("month_abbr"))
             .groupBy("month_abbr")
             .agg(count("*").alias("num_completed_studies"))
             .orderBy(expr("find_in_set(month_abbr, 'Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec')"))
            )

result_df.show()


+----------+---------------------+
|month_abbr|num_completed_studies|
+----------+---------------------+
|       Jan|                 1131|
|       Feb|                  934|
|       Mar|                 1227|
|       Apr|                  967|
|       May|                  984|
|       Jun|                 1094|
|       Jul|                  819|
|       Aug|                  700|
|       Sep|                  528|
|       Oct|                  187|
+----------+---------------------+



In [0]:
spark.catalog.listTables()

Out[14]: [Table(name='clinicaltrial_2021', catalog='spark_catalog', namespace=['default'], description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='pharma', catalog='spark_catalog', namespace=['default'], description=None, tableType='EXTERNAL', isTemporary=False)]